In [1]:
from enum import Enum

class TestModel(Enum):
    clinical_mobilenet_sum = "lesion-detection_val_ap_0_1542_test_ap_0_1294_epoch60_05-17-2023 21-23-17_clinical_mobilenet_sum"

In [2]:
import os, pickle

In [3]:
model_select = TestModel.clinical_mobilenet_sum

In [4]:
from models.train import TrainingInfo

with open(os.path.join("training_records", f"{model_select.value}.pkl"), "rb") as f:
    train_info: TrainingInfo = pickle.load(f)

In [6]:
from models.build import create_model_from_setup

model = create_model_from_setup(train_info.model_setup)
model.to('cpu')

Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3


c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ExtractFusePerform(
  (feature_extractors): ModuleDict(
    (xrays): ImageFeatureExtractor(
      (backbone): Sequential(
        (0): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): Hardswish()
          )
          (1): InvertedResidual(
            (block): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
                (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
              )
              (1): SqueezeExcitation(
                (avgpool): AdaptiveAvgPool2d(output_size=1)
                (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
                (fc2): Conv2d(8, 16, k

In [7]:
from typing import Dict
import torch
device= 'cpu'
cp: Dict = torch.load(
        os.path.join("trained_models", model_select.value), map_location=device
    )

In [12]:
xray_dict = {k:v  for k,v  in cp['model_state_dict'].items() if k.startswith("feature_extractors.xrays")}
    

In [13]:
xray_dict.keys()

dict_keys(['feature_extractors.xrays.backbone.0.0.0.weight', 'feature_extractors.xrays.backbone.0.0.1.weight', 'feature_extractors.xrays.backbone.0.0.1.bias', 'feature_extractors.xrays.backbone.0.0.1.running_mean', 'feature_extractors.xrays.backbone.0.0.1.running_var', 'feature_extractors.xrays.backbone.0.0.1.num_batches_tracked', 'feature_extractors.xrays.backbone.0.1.block.0.0.weight', 'feature_extractors.xrays.backbone.0.1.block.0.1.weight', 'feature_extractors.xrays.backbone.0.1.block.0.1.bias', 'feature_extractors.xrays.backbone.0.1.block.0.1.running_mean', 'feature_extractors.xrays.backbone.0.1.block.0.1.running_var', 'feature_extractors.xrays.backbone.0.1.block.0.1.num_batches_tracked', 'feature_extractors.xrays.backbone.0.1.block.1.fc1.weight', 'feature_extractors.xrays.backbone.0.1.block.1.fc1.bias', 'feature_extractors.xrays.backbone.0.1.block.1.fc2.weight', 'feature_extractors.xrays.backbone.0.1.block.1.fc2.bias', 'feature_extractors.xrays.backbone.0.1.block.2.0.weight', 'fe

In [21]:
next(model.parameters()).device

device(type='cpu')

In [16]:
model.load_state_dict(xray_dict, strict=False)

_IncompatibleKeys(missing_keys=['feature_extractors.clinical.backbone.0.0.0.weight', 'feature_extractors.clinical.backbone.0.0.1.weight', 'feature_extractors.clinical.backbone.0.0.1.bias', 'feature_extractors.clinical.backbone.0.0.1.running_mean', 'feature_extractors.clinical.backbone.0.0.1.running_var', 'feature_extractors.clinical.backbone.0.1.block.0.0.weight', 'feature_extractors.clinical.backbone.0.1.block.0.1.weight', 'feature_extractors.clinical.backbone.0.1.block.0.1.bias', 'feature_extractors.clinical.backbone.0.1.block.0.1.running_mean', 'feature_extractors.clinical.backbone.0.1.block.0.1.running_var', 'feature_extractors.clinical.backbone.0.1.block.1.fc1.weight', 'feature_extractors.clinical.backbone.0.1.block.1.fc1.bias', 'feature_extractors.clinical.backbone.0.1.block.1.fc2.weight', 'feature_extractors.clinical.backbone.0.1.block.1.fc2.bias', 'feature_extractors.clinical.backbone.0.1.block.2.0.weight', 'feature_extractors.clinical.backbone.0.1.block.2.1.weight', 'feature_e

In [24]:
for n, param in model.named_parameters():
    if n.startswith("feature_extractors.xrays"):
        param.requires_grad = False

In [25]:
load_part_params = list(filter(lambda p: p.requires_grad, [ param for n, param in model.named_parameters() if n.startswith("feature_extractors.xrays")]))

In [26]:
load_part_params

[]

In [34]:
def load_cl_pretrained(model, model_name, load_part, fix_weight):
    # set load_part = "feature_extractors.xrays" to load the pretrained image backbone.
    device = next(model.parameters()).device

    cp: Dict = torch.load(
        os.path.join("trained_models", model_name), map_location=device
    )

    load_part_dict = {k:v  for k,v  in cp['model_state_dict'].items() if k.startswith(load_part)}
    model.load_state_dict(load_part_dict, strict=False)

    if fix_weight:
        for n, param in model.named_parameters():
            if n.startswith(load_part):
                param.requires_grad = False
        load_part_params = list(filter(lambda p: p.requires_grad, [ param for n, param in model.named_parameters() if n.startswith(load_part)]))
        assert len(load_part_params)==0
    
    return model

In [35]:
class CLPretrainedLoadParams:
    def __init__(self, model_name, model_part, fix_weight) -> None:
        self.model_name = model_name
        self.model_part = model_part
        self.fix_wieght = fix_weight

In [36]:
cl_pretrained_load_params = [
    CLPretrainedLoadParams(
        model_name = "lesion-detection_val_ap_0_1542_test_ap_0_1294_epoch60_05-17-2023 21-23-17_clinical_mobilenet_sum",
        model_part = "feature_extractors.xrays",
        fix_weight = True       
    )
]

In [37]:
for cl_param in cl_pretrained_load_params:
    model = load_cl_pretrained(model, cl_param.model_name, cl_param.model_part, cl_param.fix_wieght)

In [28]:
load_cl_pretrained(model_select, model, "feature_extractors.xrays", fix_weight=True)

ExtractFusePerform(
  (feature_extractors): ModuleDict(
    (xrays): ImageFeatureExtractor(
      (backbone): Sequential(
        (0): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): Hardswish()
          )
          (1): InvertedResidual(
            (block): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
                (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
              )
              (1): SqueezeExcitation(
                (avgpool): AdaptiveAvgPool2d(output_size=1)
                (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
                (fc2): Conv2d(8, 16, k

In [ ]:
def get_trained_model(
    model_select, device,
) -> Tuple[nn.Module, TrainingInfo, Union[Optimizer, None]]:

    with open(os.path.join("training_records", f"{model_select.value}.pkl"), "rb") as f:
        train_info: TrainingInfo = pickle.load(f)

    model = create_model_from_setup(train_info.model_setup)
    model.to(device)

    cp: Dict = torch.load(
        os.path.join("trained_models", model_select.value), map_location=device
    )

    model.load_state_dict(cp["model_state_dict"])

    model.to(device)

    dynamic_loss_weight = None
    if "dynamic_weight_state_dict" in cp:
        dynamic_loss_weight = get_dynamic_loss(loss_keys=model.get_all_losses_keys(), device=device)
        dynamic_loss_weight.to(device)
        dynamic_loss_weight.load_state_dict(cp["dynamic_weight_state_dict"])

    params = model.get_all_params(dynamic_loss_weight=dynamic_loss_weight)

    optim = None
    if "optimizer_state_dict" in cp:
        optim: torch.optim.optimizer.Optimizer = get_optimiser(
            params, train_info.model_setup
        )
        optim.load_state_dict(cp["optimizer_state_dict"])

    return model, train_info, optim, dynamic_loss_weight
    # return model, train_info, None, None